In [2]:
import torch

In [14]:
batch = 4
seq_len = 2
dim = 2
a=torch.rand(batch,seq_len,dim)
print(a)
b=torch.rand(batch,1,dim)
print(b)
res = a + b
print(res)

tensor([[[0.9524, 0.3706],
         [0.1303, 0.7374]],

        [[0.9732, 0.3188],
         [0.9263, 0.7869]],

        [[0.8464, 0.7628],
         [0.8697, 0.5228]],

        [[0.0322, 0.2308],
         [0.3270, 0.5727]]])
tensor([[[0.6635, 0.5353]],

        [[0.4635, 0.8658]],

        [[0.6650, 0.5835]],

        [[0.6548, 0.4669]]])
tensor([[[1.6159, 0.9060],
         [0.7938, 1.2727]],

        [[1.4367, 1.1845],
         [1.3898, 1.6527]],

        [[1.5115, 1.3463],
         [1.5348, 1.1062]],

        [[0.6869, 0.6977],
         [0.9818, 1.0396]]])
